我有一个期货数据10分钟粒度的csv，格式如下 
```
timestamps,open,high,low,close,volume,amount 
2025-07-21 21:20:00,2323.0,2324.0,2321.0,2323.0,16579.0,385041430.0 
```
交易时间是 
1. 上午：9:00 - 11:30 
2. 下午：13:30 - 15:00； 
3. 晚上：21:00 - 23:00 

我想统计上午、下午、晚上开盘后半小时内的和开盘后半小时外的波动率是否有显著差异。

## 🧩 一、问题目标

比较期货在 **开盘后半小时内** 与 **开盘后半小时外** 的波动率是否有显著差异，分别针对：

* 上午盘：09:00 - 11:30
* 下午盘：13:30 - 15:00
* 夜盘：21:00 - 23:00

## 🧮 二、关键定义

### 1. 波动率的定义（可选几种）

我们可以选择：

* **收盘价收益率标准差**：
  $$
  \sigma = \text{std}(\ln(P_t / P_{t-1}))
  $$
* 或者使用 **高低价波动率**：
  $$
  \sigma = \ln(\text{high}) - \ln(\text{low})
  $$
  通常建议用 **收益率标准差**，更稳定。

### 2. 前后半小时的区间定义（10分钟K线）

每个交易时段按10分钟划分：

| 时段  | 开盘后半小时内        | 开盘后半小时外        |
| --- | ------------- | ------------- |
| 上午盘 | 09:00 - 09:30 | 09:30 - 11:30 |
| 下午盘 | 13:30 - 14:00 | 14:00 - 15:00 |
| 夜盘  | 21:00 - 21:30 | 21:30 - 23:00 |


## 🧠 三、实现思路（pandas 实现）

假设文件名为 `data/C9999.XDCE.10m.20251018.csv`。

### 步骤 1：加载并预处理

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/C9999.XDCE.10m.20251018.csv", parse_dates=["timestamps"])
df = df.sort_values("timestamps")

In [ ]:
df['close'].plot()

### 步骤 2：计算收益率

In [ ]:
df["ret"] = np.log(df["close"]).diff()

In [ ]:
df["ret"].plot()

极值处理

In [ ]:
q_low, q_high = df["ret"].quantile(0.01), df["ret"].quantile(0.99)
df["ret"] = df["ret"].clip(q_low, q_high)

In [ ]:
df["ret"].plot()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
df["ret"].hist(bins=100, alpha=0.6, label="Winsorized ret")
plt.legend()
plt.title("Distribution of Returns after Outlier Clipping")
plt.show()

### 步骤 3：提取交易日、时段

In [ ]:
df["date"] = df["timestamps"].dt.date
df["time"] = df["timestamps"].dt.time

In [ ]:
df.head()

## ⏰ 四、定义时段函数

In [ ]:
from datetime import time

def get_session_period(t):
    if time(9,0) <= t < time(11,30):
        return "morning"
    elif time(13,30) <= t < time(15,0):
        return "afternoon"
    elif time(21,0) <= t < time(23,0):
        return "night"
    else:
        return None

In [ ]:
df["session"] = df["timestamps"].dt.time.apply(get_session_period)

In [ ]:
df['session'].value_counts()

## 📊 五、提取开盘前后半小时，所有交易日循环统计

In [ ]:
periods = {
    "morning": {
        "before": (time(9,0), time(9,30)),
        "after": (time(9,30), time(11,30)),
    },
    "afternoon": {
        "before": (time(13,30), time(14,0)),
        "after": (time(14,0), time(15,0)),
    },
    "night": {
        "before": (time(21,0), time(21,30)),
        "after": (time(21,30), time(23,0)),
    }
}

records = []

for d, group in df.groupby("date"):
    for session, p in periods.items():
        before_mask = (group["time"] >= p["before"][0]) & (group["time"] < p["before"][1])
        after_mask  = (group["time"] >= p["after"][0]) & (group["time"] < p["after"][1])
        
        before_vol = group.loc[before_mask, "ret"].std()
        after_vol  = group.loc[after_mask, "ret"].std()
        
        if not np.isnan(before_vol) and not np.isnan(after_vol):
            records.append({
                "date": d,
                "session": session,
                "before_vol": before_vol,
                "after_vol": after_vol
            })

vol_df = pd.DataFrame(records)

In [ ]:
vol_df

## 🧪 六、显著性检验（配对样本 t 检验）

In [ ]:
from scipy.stats import ttest_rel

for s in ["morning", "afternoon", "night"]:
    sub = vol_df[vol_df["session"] == s].dropna()
    t, p = ttest_rel(sub["before_vol"], sub["after_vol"])
    print(f"{s}：t={t:.3f}, p={p:.4f}")

> 若 p < 0.05，则认为开盘前后半小时波动率显著不同。

## 📈 七、可视化建议

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
vol_df.boxplot(column=["before_vol","after_vol"], by="session")
plt.suptitle("")
plt.title("Volatility Before vs After Open by Session")
plt.ylabel("Volatility (std of returns)")
plt.show()
